# Real Estate Price Prediction

### 1. Web Scraping
### `Open Sooq`

#### Import the Libraries

In [44]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import time
import re

In [45]:
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0"}

In [46]:
url = "https://om.opensooq.com/en/property/property-for-rent"

In [47]:
properties = {
        "Property Title": [],
        "Location": [],
        "Bedrooms": [],
        "Price": [],
        "Size": [],
        "Listing Type": []
}

In [50]:
page_num = 1
while True:
    print(f"Scraping Page {page_num}...")
    try:
        response = requests.get(url, headers = header)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"error fetching page {current_page_url}: {e}")
        break

    soup = BeautifulSoup(response.content, "html.parser")
    cards = soup.find_all("div", class_ = "postDet flex flex-1 flexWrap flexSpaceBetween flexDirectionColumn overflowHidden ripple")
    if not cards:
        print("No Property Found!")
    for card in cards:
        # Property_Title
        title = card.find("h2", class_ = "breakWord trimTwoLines font-20 me-auto")
        properties["Property Title"].append(title.text.strip() if title else None)

        # Location
        location = card.find("div", class_ = "flex alignItems gap-5 darkGrayColor")
        properties["Location"].append(location.text.strip() if location else None)

        size = None
        bedrooms = None

        p_tag = card.find("p")
        if p_tag and p_tag.text:
            text = p_tag.text.strip()

        # Surface Area
        size_match = re.search(r"Surface\s*Area:\s*([\d,]+)\s*m2", text)
        if size_match:
            size = size_match.group(1).replace(",", "")

        # Bedrooms
        bedrooms_match = re.search(r"(\d[\d+]*)\s+Bedrooms?", text)
        if bedrooms_match:
            bedrooms = bedrooms_match.group(1)

        properties["Size"].append(size)
        properties["Bedrooms"].append(bedrooms)


        # Price
        price = card.find("div", class_ = "priceColor bold alignSelfCenter font-18 ms-auto")
        properties["Price"].append(price.text.strip() if price else None)

        # Listing Type
        properties["Listing Type"].append("For Rent")

    # Scrape all paginated pages
    next_button = soup.find("a",
    class_="blueHoverBg border pagerItem flex alignItems justifyContent radius-8 ripple",
    attrs={"title": "Go to next page"},
    href=True
    )

    next_button = soup.find('a', class_='blueHoverBg border pagerItem flex alignItems justifyContent radius-8 ripple', attrs={'title':"Go to next page"}, href=True)
    if next_button:
        next_page_relative_url = next_button["href"]
        new_url = requests.compat.urljoin(url, next_page_relative_url)

    if new_url == url:
        print("Reached the last page. No more properties found!")
        break
    else:
        url = new_url
        page_num += 1
        time.sleep(1)

Scraping Page 1...
Scraping Page 2...
Scraping Page 3...
Scraping Page 4...
Scraping Page 5...
Scraping Page 6...
Scraping Page 7...
Scraping Page 8...
Scraping Page 9...
Scraping Page 10...
Scraping Page 11...
Scraping Page 12...
Scraping Page 13...
Scraping Page 14...
Scraping Page 15...
Scraping Page 16...
Scraping Page 17...
Scraping Page 18...
Scraping Page 19...
Scraping Page 20...
Scraping Page 21...
Scraping Page 22...
Scraping Page 23...
Scraping Page 24...
Scraping Page 25...
Scraping Page 26...
Scraping Page 27...
Scraping Page 28...
Scraping Page 29...
Scraping Page 30...
Scraping Page 31...
Scraping Page 32...
Scraping Page 33...
Scraping Page 34...
Scraping Page 35...
Scraping Page 36...
Scraping Page 37...
Scraping Page 38...
Scraping Page 39...
Scraping Page 40...
Scraping Page 41...
Scraping Page 42...
Scraping Page 43...
Scraping Page 44...
Scraping Page 45...
Scraping Page 46...
Scraping Page 47...
Scraping Page 48...
Scraping Page 49...
Scraping Page 50...
Scraping 

In [51]:
df = pd.DataFrame(properties)
df

,Property Title,Location,Bedrooms,Price,Size,Listing Type
0,115 m2 2 Bedrooms Apartments for Rent in Dhofa...,"Dhofar, Salala",2,45 OMR,115,For Rent
1,Furnished Daily in Dhofar Salala,"Dhofar, Salala",None,15 OMR,None,For Rent
2,900 m2 2 Bedrooms Apartments for Rent in Dhofa...,"Dhofar, Salala",2,25 OMR,900,For Rent
3,More than 6 bedrooms Chalet for Rent in Al Bat...,"Al Batinah, Suwaiq",None,70 OMR,5000,For Rent
4,60 m2 1 Bedroom Apartments for Rent in Dhofar ...,"Dhofar, Salala",1,45 OMR,60,For Rent
...,...,...,...,...,...,...
5374,1000 m2 1 Bedroom Apartments for Rent in Musca...,"Muscat, Al Khuwair",1,250 OMR,1000,For Rent
5375,90 m2 2 Bedrooms Apartments for Rent in Muscat...,"Muscat, Al Maabilah",2,180 OMR,90,For Rent
5376,85 m2 2 Bedrooms Apartments for Rent in Muscat...,"Muscat, Al Maabilah",2,185 OMR,85,For Rent
5377,20 m2 1 Bedroom Apartments for Rent in Muscat ...,"Muscat, Al Maabilah",1,85 OMR,20,For Rent


In [58]:
df.to_csv("Open_Sooq.csv", index=False)